In [ ]:
!pip install transformers accelerate torch bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name = "axondendriteplus/contract_drafter-Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
).to("cuda")  # Ensure the model is on GPU


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_name = "axondendriteplus/contract_drafter-Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
).to("cuda")  # Ensure the model is on GPU

# Gather user inputs
print("Please enter the following details for the Employment Agreement:")
company_name = input("Employer Company Name: ")
employee_name = input("Employee Name: ")
employee_address = input("Employee Address: ")
contact_number = input("Employee Contact Number: ")
employee_email = input("Employee Email: ")
employer_address = input("Employer Address: ")
employer_phone = input("Employer Phone: ")
employer_email = input("Employer Email: ")
start_date = input("Employment Start Date: ")
end_date = input("Employment End Date: ")
salary = input("Monthly Salary (in ₹): ")

# Construct the prompt with the user inputs
prompt = (
    f"Generate an Indian legal contract for: Employment Agreement between {company_name} and {employee_name}.\n\n"
    f"**Employee Details:**\n"
    f"Name: {employee_name}\n"
    f"Address: {employee_address}\n"
    f"Contact Number: {contact_number}\n"
    f"Email: {employee_email}\n\n"
    f"**Employer's Details:**\n"
    f"Company Name: {company_name}\n"
    f"Address: {employer_address}\n"
    f"Phone: {employer_phone}\n"
    f"Email: {employer_email}\n\n"
    f"**Terms & Conditions:**\n"
    f"The Employee agrees to work as a part-time/full-time employee with {company_name} starting from {start_date} until {end_date}. "
    f"Salary per month: ₹{salary}/- plus additional allowances as per company policies.\n\n"
    f"Please generate the complete legal contract in a detailed, formatted manner similar to a formal Indian Employment Agreement template."
)

# Tokenize input and move to GPU
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=2048,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

# Decode output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nGenerated Employment Contract:\n")
print(response)

# Save the contract to a text file
with open("employment_agreement.txt", "w") as f:
    f.write(response)

print("\nContract saved as employment_agreement.txt")


Please enter the following details for the Employment Agreement:
Employer Company Name: ABC Tech Solutions Pvt Ltd
Employee Name: Rahul Sharma
Employee Address: 789 Residency Road, Delhi, India
Employee Contact Number: +91 9876543210
Employee Email: rahul.sharma@email.com
Employer Address: 123 Corporate Park, Bangalore, India
Employer Phone: +91 9998887776
Employer Email: hr@abctech.com
Employment Start Date: 1st April 2025
Employment End Date: 31st March 2026
Monthly Salary (in ₹): 85,000

Generated Employment Contract:

Generate an Indian legal contract for: Employment Agreement between ABC Tech Solutions Pvt Ltd and Rahul Sharma.

**Employee Details:**
Name: Rahul Sharma
Address: 789 Residency Road, Delhi, India
Contact Number: +91 9876543210
Email: rahul.sharma@email.com

**Employer's Details:**
Company Name: ABC Tech Solutions Pvt Ltd
Address: 123 Corporate Park, Bangalore, India
Phone: +91 9998887776
Email: hr@abctech.com

**Terms & Conditions:**
The Employee agrees to work as a 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_name = "axondendriteplus/contract_drafter-Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
).to("cuda")  # Ensure the model is on GPU

# Gather user inputs for the Lease Agreement
print("Please enter the following details for the Lease Agreement:")

landlord_name = input("Landlord Name: ")
landlord_address = input("Landlord Address: ")
landlord_contact = input("Landlord Contact Number: ")
landlord_email = input("Landlord Email: ")

tenant_name = input("Tenant Name: ")
tenant_address = input("Tenant Address: ")
tenant_contact = input("Tenant Contact Number: ")
tenant_email = input("Tenant Email: ")

property_address = input("Property Address: ")
lease_start_date = input("Lease Start Date: ")
lease_end_date = input("Lease End Date: ")
monthly_rent = input("Monthly Rent (in ₹): ")
security_deposit = input("Security Deposit (in ₹): ")

# Construct the prompt with the user inputs
prompt = (
    f"Generate an Indian legal contract for: Lease Agreement between {landlord_name} (Landlord) and {tenant_name} (Tenant).\n\n"
    f"**Landlord Details:**\n"
    f"Name: {landlord_name}\n"
    f"Address: {landlord_address}\n"
    f"Contact Number: {landlord_contact}\n"
    f"Email: {landlord_email}\n\n"
    f"**Tenant Details:**\n"
    f"Name: {tenant_name}\n"
    f"Address: {tenant_address}\n"
    f"Contact Number: {tenant_contact}\n"
    f"Email: {tenant_email}\n\n"
    f"**Property Details:**\n"
    f"Property Address: {property_address}\n\n"
    f"**Lease Terms:**\n"
    f"Lease Period: From {lease_start_date} to {lease_end_date}\n"
    f"Monthly Rent: ₹{monthly_rent}\n"
    f"Security Deposit: ₹{security_deposit}\n\n"
    f"Please generate a complete and detailed legal lease agreement formatted as a formal Indian Lease Agreement template, including all essential clauses such as payment terms, maintenance responsibilities, termination conditions, and dispute resolution."
)

# Tokenize input and move to GPU
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=2048,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

# Decode output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nGenerated Lease Agreement:\n")
print(response)

# Save the contract to a text file
with open("lease_agreement.txt", "w") as f:
    f.write(response)

print("\nContract saved as lease_agreement.txt")


Please enter the following details for the Lease Agreement:
Landlord Name: Ramesh Kumar  
Landlord Address: 45 Green Avenue, Mumbai, India
Landlord Contact Number: +91 9876543210
Landlord Email: ramesh.kumar@email.com
Tenant Name: Anjali Sharma
Tenant Address: 23 Residency Road, Pune, India
Tenant Contact Number: +91 9988776655
Tenant Email: anjali.sharma@email.com
Property Address: Flat No. 302, Sunshine Apartments, Andheri, Mumbai
Lease Start Date: 1st April 2025
Lease End Date: 31st March 2026
Monthly Rent (in ₹): ₹25,000
Security Deposit (in ₹): ₹75,000

Generated Lease Agreement:

Generate an Indian legal contract for: Lease Agreement between Ramesh Kumar   (Landlord) and Anjali Sharma (Tenant).

**Landlord Details:**
Name: Ramesh Kumar  
Address: 45 Green Avenue, Mumbai, India
Contact Number: +91 9876543210
Email: ramesh.kumar@email.com

**Tenant Details:**
Name: Anjali Sharma
Address: 23 Residency Road, Pune, India
Contact Number: +91 9988776655
Email: anjali.sharma@email.com

*

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_name = "axondendriteplus/contract_drafter-Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
).to("cuda")  # Ensure the model is on GPU

# Gather user inputs for Business Agreement
print("Please enter the following details for the Business Agreement:")

company_name = input("Company Name: ")
company_address = input("Company Address: ")
partner_name = input("Partner Name: ")
partner_address = input("Partner Address: ")
service_description = input("Scope of Work and Services: ")
payment_terms = input("Payment Terms: ")
agreement_duration = input("Agreement Duration: ")
dispute_resolution = input("Dispute Resolution Method: ")
termination_clause = input("Termination Clause: ")

# Construct the prompt with user inputs
prompt = (
    f"Generate an Indian legal contract for: Business Agreement.\n\n"
    f"**Company Details:**\n"
    f"Company Name: {company_name}\n"
    f"Address: {company_address}\n\n"
    f"**Parties Involved:**\n"
    f"1. {company_name}, hereinafter referred to as 'The Company'\n"
    f"   - Registered at: {company_address}\n\n"
    f"2. {partner_name}, hereinafter referred to as 'Partner'\n"
    f"   - Partner Address: {partner_address}\n\n"
    f"**Scope of Work and Services:** {service_description}\n\n"
    f"**Payment Terms:** {payment_terms}\n\n"
    f"**Agreement Duration:** {agreement_duration}\n\n"
    f"**Dispute Resolution:** {dispute_resolution}\n\n"
    f"**Termination Clause:** {termination_clause}\n\n"
    f"Please generate a complete and detailed Indian Business Agreement, including essential clauses such as obligations, liabilities, confidentiality, force majeure, governing law, and termination conditions."
)

# Tokenize input and move to GPU
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=2048,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

# Decode output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nGenerated Business Agreement:\n")
print(response)

# Save the contract to a text file
with open("business_agreement.txt", "w") as f:
    f.write(response)

print("\nContract saved as business_agreement.txt")


Please enter the following details for the Business Agreement:
Company Name: ABC Pvt Ltd
Company Address: 101 Corporate Tower, Connaught Place, New Delhi, India
Partner Name: XYZ Consulting Services
Partner Address: 507 Business Hub, Electronic City, Bangalore, India
Scope of Work and Services: XYZ Consulting Services will provide business strategy consulting and technology implementation services for ABC Pvt Ltd, including market research, process optimization, and IT system integration.  
Payment Terms: A monthly retainer of ₹2,00,000, payable on the 5th of each month. Additional project-based fees as per mutual agreement.
Agreement Duration: 12 months, starting from 1st April 2025 to 31st March 2026.
Dispute Resolution Method: All disputes shall be resolved through arbitration under the Arbitration and Conciliation Act, 1996, in New Delhi.
Termination Clause: Either party may terminate the agreement with 60 days' written notice. Termination due to breach of contract will be immediat

In [ ]:
!pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=6f3da27bc75ec10ec4d28812f60a45284e0014c368a216dc55a609f48036548f
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:

from fpdf import FPDF

def text_to_pdf(input_text_file, output_pdf_file):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    with open(input_text_file, "r", encoding="utf-8") as file:
        for line in file:
            line = line.replace("₹", "INR")  # Fix Rupee symbol
            line = line.replace("’", "'")   # Fix curly apostrophe
            line = line.replace("“", '"').replace("”", '"')  # Fix curly quotes

            pdf.multi_cell(0, 10, txt=line.strip(), align="L")  # Use multi_cell to handle long lines

    pdf.output(output_pdf_file)
    print(f"PDF saved as {output_pdf_file}")

# Convert text file to PDF
input_text_file = "lease_agreement.txt"
output_pdf_file = "lease_contract.pdf"
text_to_pdf(input_text_file, output_pdf_file)


PDF saved as lease_contract.pdf


In [ ]:
!pip install googletrans==4.0.0-rc1

from googletrans import Translator

def translate_text_file(input_file, output_file, dest_lang="hi"):
    translator = Translator()

    # Read the English text file
    with open(input_file, "r", encoding="utf-8") as file:
        english_text = file.read()

    # Translate to Hindi
    translated_text = translator.translate(english_text, dest=dest_lang).text

    # Save the Hindi text to a new file
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(translated_text)

    print(f"Translation completed! Hindi text saved in: {output_file}")

# Example usage
input_file = "/content/employment_agreement.txt"  # Replace with your text file
output_file = "hindi_employment_agreement.txt"
translate_text_file(input_file, output_file)


TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
!pip install reportlab

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

def text_to_pdf(input_text_file, output_pdf_file):
    c = canvas.Canvas(output_pdf_file, pagesize=letter)
    c.setFont("Helvetica", 12)

    with open(input_text_file, "r", encoding="utf-8") as file:
        y = 750  # Start position for text
        for line in file:
            c.drawString(100, y, line.strip())
            y -= 20  # Move down for the next line

    c.save()
    print(f"PDF saved as {output_pdf_file}")

# Convert Hindi text file to PDF
text_to_pdf("/content/hindi_lease_agreement.txt", "hindi_lease_agreement.pdf")


PDF saved as hindi_lease_agreement.pdf
